# LDA

In [1]:
import pandas as pd
import os
import csv

# Identify the working directory and data files
working_directory = './28-LDA'

# Create the working directory if needed
try:
    os.makedirs(working_directory, exist_ok=True)
except OSError as error:
    print(f"Error creating {working_directory}: {error}")

In [2]:
%pip install --quiet --upgrade nltk gensim spacy pyldavis

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
texthero 1.1.0 requires gensim<4.0,>=3.6.0, but you have gensim 4.2.0 which is incompatible.
texthero 1.1.0 requires spacy<3.0.0, but you have spacy 3.5.1 which is incompatible.
en-core-web-sm 2.3.1 requires spacy<2.4.0,>=2.3.0, but you have spacy 3.5.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m nltk.downloader --quiet 'all'
!python -m spacy download en_core_web_sm > /dev/null

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# Read the preprocessed data into a dataframe
import pickle

with open('./21-Preprocess-Combined-Data-v2/dataframe.pickle', 'rb') as f:
    df = pickle.load(f)

In [5]:
df.shape

(5736, 20)

In [6]:
df.head(1)

,ROW_ID,FOI_TEXT,DEVICE_PROBLEM_CODE,DEVICE_PROBLEM_TEXT,GENERIC_NAME,DEVICE_REPORT_PRODUCT_CODE,UDI-DI,UDI-PUBLIC,DATE_OF_EVENT,REPORTER_OCCUPATION_CODE,REPORT_DATE,EVENT_LOCATION,SOURCE_TYPE,TOKENIZED_TEXT,NOPUNCT_TEXT,NOSTOPWORDS_TEXT,NODIGITS_TEXT,POS_TEXT,LEMMATIZED_TEXT,STEMMED_TEXT
0,1969025,IT WAS REPORTED THAT THE TRANSMITTER LOST CONN...,3283,Wireless Communication Problem,CONTINUOUS GLUCOSE MONITOR,QBJ,,,07/30/2020,000,,I,CONSUMER,"[it, was, reported, that, the, transmitter, lo...","[it, was, reported, that, the, transmitter, lo...","[reported, transmitter, lost, connection, pump...","[reported, transmitter, lost, connection, pump...","[(reported, VBN), (transmitter, NN), (lost, VB...","[report, transmitter, lose, connection, pump, ...","[report, transmitt, lost, connect, pump, great..."


In [7]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = df['FOI_TEXT'].tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['it', 'was', 'reported', 'that', 'the', 'transmitter', 'lost', 'connection', 'with', 'the', 'pump', 'for', 'greater', 'than', 'hour', 'the', 'transmitter', 'ultimately', 'regained', 'connection', 'with', 'the', 'pump', 'no', 'additional', 'patient', 'or', 'event', 'information', 'was']


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


['report', 'transmitter', 'lose', 'connection', 'pump', 'great', 'hour', 'transmitter', 'ultimately', 'regain', 'connection', 'pump', 'additional', 'patient', 'event', 'information', 'available']


In [11]:
data_words_bigrams[0]

['reported',
 'transmitter',
 'lost',
 'connection',
 'pump',
 'greater',
 'hour',
 'transmitter',
 'ultimately',
 'regained',
 'connection',
 'pump',
 'additional',
 'patient',
 'event',
 'information',
 'available']

In [12]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 2), (13, 1)]


In [13]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [14]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.102*"report" + 0.079*"transmitter" + 0.066*"fail" + 0.057*"error" + '
  '0.056*"evaluate" + 0.051*"cause" + 0.051*"probable" + 0.051*"allegation" + '
  '0.050*"confirm" + 0.050*"intervention"'),
 (1,
  '0.092*"transmitter" + 0.069*"pump" + 0.067*"connection" + 0.059*"patient" + '
  '0.052*"report" + 0.051*"additional" + 0.051*"information" + '
  '0.048*"available" + 0.048*"event" + 0.045*"lose"'),
 (2,
  '0.055*"report" + 0.043*"system" + 0.043*"device" + 0.037*"loss" + '
  '0.036*"support" + 0.028*"share" + 0.027*"sensor" + 0.027*"mobile" + '
  '0.027*"signal" + 0.025*"relate"'),
 (3,
  '0.081*"loss" + 0.071*"signal" + 0.070*"report" + 0.054*"perform" + '
  '0.043*"hour" + 0.036*"window" + 0.036*"find" + 0.036*"allegation" + '
  '0.036*"probable" + 0.036*"cause"'),
 (4,
  '0.077*"bg" + 0.073*"glucose" + 0.068*"reading" + 0.066*"read" + '
  '0.039*"meter" + 0.039*"reportedly" + 0.036*"blood" + 0.035*"customer" + '
  '0.035*"monitor" + 0.035*"continuous"'),
 (5,
  '0.085*"repo

In [15]:
from gensim.models import CoherenceModel

# Compute Coherence Score
# TODO: Pickle these as the baseline models
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.6670688888105576


In [16]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    # TODO: Pickle each lda_model and coherence_model_lda
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [17]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 1
max_topics = 25
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {
    'Validation_Set': [],
    'Topics': [],
    'Alpha': [],
    'Beta': [],
    'Coherence': []
}

# Can take a long time to run
if True:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv(f"{working_directory}/lda_tuning_results.csv", index=False)
    pbar.close()

 39%|███▉      | 568/1440 [48:22<59:12,  4.07s/it]  /home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))
100%|██████████| 1440/1440 [2:05:26<00:00,  5.23s/it] 


In [18]:
model_results_df = pd.DataFrame(model_results)

In [19]:
import pickle
import datetime

TAG = f"{datetime.datetime.now():%Y-%m-%d-%s}"

with open(f"{working_directory}/model_results_df-{max_topics}-topics-{TAG}.pickle", 'wb') as f:
    pickle.dump(model_results_df, f)

In [20]:
# TODO: Graph the coherence scores based on some common criteria
model_results_df

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,1,0.01,0.01,0.573277
1,75% Corpus,1,0.01,0.31,0.573277
2,75% Corpus,1,0.01,0.61,0.573277
3,75% Corpus,1,0.01,0.91,0.573277
4,75% Corpus,1,0.01,symmetric,0.573277
...,...,...,...,...,...
1435,100% Corpus,24,asymmetric,0.01,0.554398
1436,100% Corpus,24,asymmetric,0.31,0.565456
1437,100% Corpus,24,asymmetric,0.61,0.553081
1438,100% Corpus,24,asymmetric,0.91,0.545392


In [26]:
num_topics = 25
alpha      = 0.01
beta       = 0.9

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           eta=beta)

In [27]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join(f"{working_directory}/ldavis_tuned_{num_topics}-topics-{TAG}.pickle")

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if True:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, f"{working_directory}/ldavis_tuned_{num_topics}-topics-{TAG}"+ str(num_topics) +'.html')

LDAvis_prepared

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.369219 -0.009682       1        1  21.866172
17     0.335785 -0.090977       2        1  20.052087
14     0.336991 -0.060511       3        1  11.104007
23     0.235859 -0.077755       4        1   9.484718
5      0.196591 -0.034817       5        1   6.214779
13     0.020660  0.261226       6        1   6.122471
19     0.064646  0.197037       7        1   6.090358
9      0.067951  0.273087       8        1   5.341722
10    -0.039387  0.186385       9        1   2.708988
22    -0.030684 -0.012393      10        1   1.966296
12     0.024778 -0.133992      11        1   1.917894
6     -0.047063  0.009016      12        1   1.624959
1     -0.064952  0.164191      13        1   1.604602
7      0.091172 -0.120528      14        1   1.533176
2     -0.076135 -0.072034      15        1   0.898657
15    -0.166111  0.000726      16        1   0.680838
21    -0.091264 -0.055224      17        1   0.262962
24    -0.160276 -0.046737      18        1   0.110524
20    -0.169372 -0.058177      19        1   0.092218
16    -0.151260 -0.054630      20        1   0.062720
0     -0.162862 -0.049781      21        1   0.059083
18    -0.157697 -0.050598      22        1   0.057127
4     -0.153940 -0.051811      23        1   0.053214
8     -0.150924 -0.056740      24        1   0.045215
3     -0.121723 -0.055282      25        1   0.045215, topic_info=              Term         Freq        Total Category  logprob  loglift
12     transmitter  3584.000000  3584.000000  Default  30.0000  30.0000
17           datum  2798.000000  2798.000000  Default  29.0000  29.0000
36         perform  2257.000000  2257.000000  Default  28.0000  28.0000
22            loss  2905.000000  2905.000000  Default  27.0000  27.0000
150         sensor  1158.000000  1158.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
202           read     0.102864   514.598072  Topic25  -6.3087  -0.8162
126        glucose     0.110738  1080.694776  Topic25  -6.2349  -1.4845
34   investigation     0.112157  1385.780760  Topic25  -6.2222  -1.7204
150         sensor     0.109457  1158.049826  Topic25  -6.2465  -1.5652
31        evaluate     0.109664  1489.975983  Topic25  -6.2447  -1.8154

[1295 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
248       1  0.073019  abdoman
248       2  0.073019  abdoman
248       3  0.073019  abdoman
248       4  0.073019  abdoman
248       5  0.073019  abdoman
...     ...       ...      ...
173      12  0.004196     zone
173      13  0.004196     zone
173      14  0.004196     zone
173      15  0.004196     zone
173      16  0.004196     zone

[8109 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 18, 15, 24, 6, 14, 20, 10, 11, 23, 13, 7, 2, 8, 3, 16, 22, 25, 21, 17, 1, 19, 5, 9, 4])

In [28]:
import os
import fnmatch
import subprocess

bucket = "praxis-2023-html-output"
website = f"http://{bucket}.s3-website-us-west-2.amazonaws.com"

# Use the fnmatch module to find all files in the current directory that end in ".html"
file_list = []
for root, dirnames, filenames in os.walk("."):
    for filename in fnmatch.filter(filenames, '*.html'):
        file_list.append(os.path.join(root, filename))

# Sort the file list alphabetically
file_list.sort()

# Create the HTML file and write the header
with open(os.path.join(".", 'index.html'), 'w') as f:
    f.write('''<html>
        <head>
            <title>Praxis 2023 HTML Output</title>
            <style>
                table {
                    border-collapse: collapse;
                    width: 100%;
                }
                th, td {
                    text-align: left;
                    padding: 8px;
                }
                th {
                    background-color: #007bff;
                    color: #fff;
                    font-weight: bold;
                }
                tr:nth-child(even) {
                    background-color: #f2f2f2;
                }
                tr:hover {
                    background-color: #ddd;
                }
            </style>
        </head>
        <body>
            <table>
                <tr><th>Name</th><th>Size</th></tr>\n
    ''')

    # Loop through each file and add a row to the table
    for file_name in file_list:
        if file_name in ['./index.html']:
            continue
            
        file_size = os.path.getsize(file_name)
        f.write(f'<tr><td><a href="{website}/{file_name}" target="_blank" rel="noopener noreferrer">{file_name}</a></td><td>{int(file_size / 1048576)} MB</td></tr>\n')

    # Write the footer and close the file
    f.write('</table></body></html>')

command = ["aws", "s3", "sync", ".", f"s3://{bucket}", "--exclude", "*", "--include", "*.html", "--no-progress"]

# Run the command and wait for it to complete
output = subprocess.run(command, capture_output=True, text=True)

# Print the output
print(output.stdout)
print('fin')

upload: ./index.html to s3://praxis-2023-html-output/index.html
upload: 28-LDA/ldavis_tuned_8-topics-2023-03-30-16802169598.html to s3://praxis-2023-html-output/28-LDA/ldavis_tuned_8-topics-2023-03-30-16802169598.html
upload: 28-LDA/ldavis_tuned_25-topics-2023-03-30-168021695925.html to s3://praxis-2023-html-output/28-LDA/ldavis_tuned_25-topics-2023-03-30-168021695925.html

fin
